
Currently we cannot create the full regridding weights for CM2.6 on the cloud due to a lack of memory. 

This is a tempoary patch. 

I did the following:
- [x] Created the weights on gyre (needed about 32GB ram)
- [x] Manually downloaded the nc file and uploaded it here
- [x] Saved it out to a zarr store on the cloud
- [ ] Reused the regridder by passing an xr.Dataset

In [1]:
import xarray as xr

In [2]:
ds = xr.open_dataset('CM26_atmos2ocean_weights.nc')
# ds = ds.chunk({'n_s':1e7}) #DO NOT DO THIS! IT will lead to horrible dask graphs!
ds

<xarray.Dataset>
Dimensions:  (n_s: 51235200)
Dimensions without coordinates: n_s
Data variables:
    S        (n_s) float64 ...
    col      (n_s) int32 ...
    row      (n_s) int32 ...

In [3]:
import os
PANGEO_SCRATCH = os.environ['PANGEO_SCRATCH']
# -> gs://pangeo-scratch/<username>
import fsspec
mapper = fsspec.get_mapper(f'{PANGEO_SCRATCH}/jbusecke/test_cm26_xesmf_weights_cloud.zarr')
# mapper can now be to read / write zarr stores

In [4]:
mapper.root

'pangeo-integration-te-3eea-prod-scratch-bucket/jbusecke/jbusecke/test_cm26_xesmf_weights_cloud.zarr'

In [11]:
ds.chunk({'n_s':-1}).to_zarr(mapper, mode='w')

[########################################] | 100% Completed |  1.9s


## Try out the cloud store

In [12]:
ds_cloud = xr.open_zarr(mapper)
ds_cloud

<xarray.Dataset>
Dimensions:  (n_s: 51235200)
Dimensions without coordinates: n_s
Data variables:
    S        (n_s) float64 dask.array<chunksize=(51235200,), meta=np.ndarray>
    col      (n_s) int32 dask.array<chunksize=(51235200,), meta=np.ndarray>
    row      (n_s) int32 dask.array<chunksize=(51235200,), meta=np.ndarray>

In [13]:
from dask_gateway import Gateway
gateway = Gateway()

/srv/conda/envs/notebook/lib/python3.9/site-packages/dask_gateway/client.py:21: FutureWarning: format_bytes is deprecated and will be removed in a future release. Please use dask.utils.format_bytes instead.
  from distributed.utils import LoopRunner, format_bytes


In [29]:
clusters = gateway.list_clusters()
for cluster in clusters:
    gateway.stop_cluster(cluster.name)

In [18]:
cluster = clusters[0]

'gateway://traefik-prod-dask-gateway.prod:80/prod.95e6a2c958774971a291601c38b9a727'

In [26]:
gateway.get_cluster(cluster.name).close()

AttributeError: 'ClusterReport' object has no attribute 'close'

In [27]:
cluster

ClusterReport<name=prod.6a6c46397ba245558f7848e5eaa14498, status=RUNNING>

In [28]:
gateway.stop_cluster(cluster.name)